In [6]:
!pip install ucimlrepo shap --quiet
!pip install --upgrade cubist --quiet
!pip install --upgrade optuna --quiet

In [7]:
from ucimlrepo import fetch_ucirepo
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
import shap
from cubist import Cubist

# fetch dataset
steel_industry_energy_consumption = fetch_ucirepo(id=851)

In [8]:
df = pd.DataFrame(steel_industry_energy_consumption.data.original)

In [9]:
df.sample(n=10)

,date,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
4108,12/02/2018 19:15,32.08,0.04,10.40,0.01,100.00,95.13,69300,Weekday,Monday,Maximum_Load
5052,22/02/2018 15:15,64.30,21.56,0.00,0.03,94.81,100.00,54900,Weekday,Thursday,Medium_Load
30680,16/11/2018 14:15,51.37,24.41,0.00,0.02,90.32,100.00,51300,Weekday,Friday,Medium_Load
9676,11/04/2018 19:15,77.87,21.82,0.00,0.04,96.29,100.00,69300,Weekday,Wednesday,Medium_Load
14773,03/06/2018 21:30,3.17,0.00,16.52,0.00,100.00,18.85,77400,Weekend,Sunday,Light_Load
13814,24/05/2018 21:45,8.39,0.00,22.79,0.00,100.00,34.55,78300,Weekday,Thursday,Medium_Load
17709,04/07/2018 11:30,108.22,56.05,0.00,0.05,88.80,100.00,41400,Weekday,Wednesday,Maximum_Load
26697,06/10/2018 02:30,2.92,6.08,0.00,0.00,43.29,100.00,9000,Weekend,Saturday,Light_Load
31343,23/11/2018 12:00,66.42,29.38,0.29,0.03,91.45,100.00,43200,Weekday,Friday,Maximum_Load
29859,08/11/2018 01:00,3.67,6.16,0.00,0.00,51.18,100.00,3600,Weekday,Thursday,Light_Load


In [10]:
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y %H:%M')

In [11]:
pd.DataFrame(steel_industry_energy_consumption.variables)

,name,role,type,demographic,description,units,missing_values
0,date,Other,Date,None,None,None,no
1,Usage_kWh,Feature,Continuous,None,Industry Energy Consumption,kWh,no
2,Lagging_Current_Reactive.Power_kVarh,Feature,Continuous,None,None,kVarh,no
3,Leading_Current_Reactive_Power_kVarh,Feature,Continuous,None,None,kVarh,no
4,CO2(tCO2),Feature,Continuous,None,None,ppm,no
5,Lagging_Current_Power_Factor,Feature,Continuous,None,None,%,no
6,Leading_Current_Power_Factor,Feature,Continuous,None,None,%,no
7,NSM,Feature,Integer,None,None,s,no
8,WeekStatus,Feature,Categorical,None,Weekend (0) or a Weekday(1),None,no
9,Day_of_week,Feature,Categorical,None,"Sunday, Monday, ..., Saturday",None,no


In [12]:
df.dtypes

date                                    datetime64[ns]
Usage_kWh                                      float64
Lagging_Current_Reactive.Power_kVarh           float64
Leading_Current_Reactive_Power_kVarh           float64
CO2(tCO2)                                      float64
Lagging_Current_Power_Factor                   float64
Leading_Current_Power_Factor                   float64
NSM                                              int64
WeekStatus                                      object
Day_of_week                                     object
Load_Type                                       object
dtype: object

In [13]:
# Crear nuevas columnas descomponiendo la fecha y hora en sus componentes
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['hour'] = df['date'].dt.hour
df['minute'] = df['date'].dt.minute
df['second'] = df['date'].dt.second
df['dayofweek'] = df['date'].dt.dayofweek  # Lunes=0, Domingo=6
df['dayofyear'] = df['date'].dt.dayofyear
df['weekofyear'] = df['date'].dt.isocalendar().week
df['quarter'] = df['date'].dt.quarter

In [14]:
df = pd.get_dummies(df, columns=["Day_of_week", "Load_Type"], drop_first=True)

In [15]:
df.columns.to_list()

['date',
 'Usage_kWh',
 'Lagging_Current_Reactive.Power_kVarh',
 'Leading_Current_Reactive_Power_kVarh',
 'CO2(tCO2)',
 'Lagging_Current_Power_Factor',
 'Leading_Current_Power_Factor',
 'NSM',
 'WeekStatus',
 'year',
 'month',
 'day',
 'hour',
 'minute',
 'second',
 'dayofweek',
 'dayofyear',
 'weekofyear',
 'quarter',
 'Day_of_week_Monday',
 'Day_of_week_Saturday',
 'Day_of_week_Sunday',
 'Day_of_week_Thursday',
 'Day_of_week_Tuesday',
 'Day_of_week_Wednesday',
 'Load_Type_Maximum_Load',
 'Load_Type_Medium_Load']

In [16]:
df['IsWeekend'] = df['WeekStatus'] == 'Weekend'

In [17]:
df = df.drop(['date', 'WeekStatus'], axis=1)

In [18]:
df.dtypes

Usage_kWh                               float64
Lagging_Current_Reactive.Power_kVarh    float64
Leading_Current_Reactive_Power_kVarh    float64
CO2(tCO2)                               float64
Lagging_Current_Power_Factor            float64
Leading_Current_Power_Factor            float64
NSM                                       int64
year                                      int32
month                                     int32
day                                       int32
hour                                      int32
minute                                    int32
second                                    int32
dayofweek                                 int32
dayofyear                                 int32
weekofyear                               UInt32
quarter                                   int32
Day_of_week_Monday                         bool
Day_of_week_Saturday                       bool
Day_of_week_Sunday                         bool
Day_of_week_Thursday                    

In [19]:
X = df
X = df.drop('Usage_kWh', axis=1)
y = df['Usage_kWh']

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de regresión lineal
model = LinearRegression()

# Entrenar el modelo
model.fit(X_train, y_train)

# Realizar predicciones
y_pred = model.predict(X_test)

# Evaluar el modelo
# Calcular MSE
mse = mean_squared_error(y_test, y_pred)

# Calcular RMSE
rmse = np.sqrt(mse)

# Calcular MAE
mae = mean_absolute_error(y_test, y_pred)

# Calcular R^2
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'R^2 Score: {r2}')

Mean Squared Error (MSE): 17.717200552024053
Root Mean Squared Error (RMSE): 4.209180508367877
Mean Absolute Error (MAE): 2.595219175169306
R^2 Score: 0.984413783416719


Agregamos modelos a utilizar, Arbol de Decisión, RandomForest, XGBoost Regressor, CUBIST

In [21]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

modelos = [('LinearRegression',LinearRegression()),
    ('DecisionTreeRegressor', DecisionTreeRegressor(max_depth=2)),
    ('RandomForestRegressor', RandomForestRegressor(min_samples_split=2, min_weight_fraction_leaf=0.0,n_estimators=40, n_jobs=-1, oob_score=True,random_state=None, verbose=0, warm_start=False)),
    ('XGBRegressor', xgb.XGBRegressor(learning_rate=0.01,n_estimators=500,max_depth=5,eval_metric='rmsle')),
    ('Cubist', Cubist(n_rules=500,neighbors=None,unbiased=True,auto=False,extrapolation=0.1,n_committees=5))
]


#procedimiento para comparar los modelos
for nombre, model in modelos:
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  mse = mean_squared_error(y_test, y_pred)
  rmse = np.sqrt(mse)
  mae = mean_absolute_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)

  print(f'{nombre}," score: ",{model.score(X_test,y_pred):.03f}',end=" ")
  print(f'Mean Squared Error (MSE): {mse:.03f}',end=" ")
  print(f'Root Mean Squared Error (RMSE): {rmse:.03f}',end=" ")
  print(f'Mean Absolute Error (MAE): {mae:.03f}',end=" ")
  print(f'R^2 Score: {r2:.03f}',end=" ")
  print("")

LinearRegression," score: ",1.000 Mean Squared Error (MSE): 17.717 Root Mean Squared Error (RMSE): 4.209 Mean Absolute Error (MAE): 2.595 R^2 Score: 0.984 
DecisionTreeRegressor," score: ",1.000 Mean Squared Error (MSE): 65.380 Root Mean Squared Error (RMSE): 8.086 Mean Absolute Error (MAE): 4.597 R^2 Score: 0.942 
RandomForestRegressor," score: ",1.000 Mean Squared Error (MSE): 1.285 Root Mean Squared Error (RMSE): 1.134 Mean Absolute Error (MAE): 0.390 R^2 Score: 0.999 
XGBRegressor," score: ",1.000 Mean Squared Error (MSE): 5.291 Root Mean Squared Error (RMSE): 2.300 Mean Absolute Error (MAE): 1.352 R^2 Score: 0.995 
Cubist," score: ",1.000 Mean Squared Error (MSE): 0.107 Root Mean Squared Error (RMSE): 0.327 Mean Absolute Error (MAE): 0.064 R^2 Score: 1.000 


Buscamos mejores valores de hiperparámetros para los 2 modelos con mejores resultados, XGBRegressor y Cubist
1. XGBRegressor

In [22]:
from pprint import pprint
import numpy as np

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_leaves = [0, 8, 10, 12, 16, 20]  # 0 equivale a 'no limit'
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)] + [None]


random_grid = {'n_estimators': n_estimators,
               'max_leaves': max_leaves,
               'max_depth': max_depth}

print('Los valores a probar en la búsqueda aleatoria son:')
pprint(random_grid)

print()
print('El número total de combinaciones de parámetros de entrenamiento es',
      len(random_grid['n_estimators']) *
      len(random_grid['max_leaves']) *
      len(random_grid['max_depth'])
      )

Los valores a probar en la búsqueda aleatoria son:
{'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_leaves': [0, 8, 10, 12, 16, 20],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

El número total de combinaciones de parámetros de entrenamiento es 720


Usaremos la recomendación de busqueda con el 10%

In [23]:
from sklearn.model_selection import RandomizedSearchCV

xgbb = xgb.XGBRegressor()
xgb_random = RandomizedSearchCV(estimator = xgbb,
                               param_distributions = random_grid,
                               n_iter = 20,
                               cv = 3,          # Validación cruzada 3-fold
                               verbose=2,
                               random_state=0,
                               n_jobs = -1      # Paralelizar en todos los cores disponibles
                               )
xgb_random.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


RandomizedSearchCV(cv=3,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None,
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None,
                                          random_state=None, ...),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_leaves': [0, 8, 10, 12, 16, 20],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=0, verbose=2)

In [24]:
xgb_random_best = xgb_random.best_estimator_

print('Los hiperparámetros del mejor modelo son:')
pprint(xgb_random.best_params_)
print()

print('Exactitud luego de búsqueda aleatoria en entrenamiento:', xgb_random_best.score(X_train, y_train))
print('Exactitud luego de búsqueda aleatoria en validación:', xgb_random_best.score(X_test, y_test))

Los hiperparámetros del mejor modelo son:
{'max_depth': 10, 'max_leaves': 8, 'n_estimators': 1000}

Exactitud luego de búsqueda aleatoria en entrenamiento: 0.9997227087327877
Exactitud luego de búsqueda aleatoria en validación: 0.9993498721698906


2. Búsqueda de hiperparámetros para el método Cubist

In [25]:
from pprint import pprint
import numpy as np

###Hiperparámetros a considerar
# n_rules (int, default=500)
# n_committees (int, default=0):
# neighbors (int, default=None)
# extrapolation (float, default=0.05):

n_rules = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 10)]
n_committees = [1, 2, 5, 10, 15]  # 5 es el recomendado en la documentación
neighbors = [int(x) for x in np.linspace(1, 9, num = 1)]
extrapolation = [0.01, 0.03, 0.05, 0.07, 0.09] #0,05 = 5% es el recomendado


random_grid = {'n_rules': n_rules,
               'n_committees': n_committees,
               'neighbors': neighbors,
               'extrapolation':extrapolation}

print('Los valores a probar en la búsqueda aleatoria son:')
pprint(random_grid)

print()
print('El número total de combinaciones de parámetros de entrenamiento es',
      len(random_grid['n_rules']) *
      len(random_grid['n_committees']) *
      len(random_grid['neighbors']) *
      len(random_grid['extrapolation'])
      )

Los valores a probar en la búsqueda aleatoria son:
{'extrapolation': [0.01, 0.03, 0.05, 0.07, 0.09],
 'n_committees': [1, 2, 5, 10, 15],
 'n_rules': [200, 288, 377, 466, 555, 644, 733, 822, 911, 1000],
 'neighbors': [1]}

El número total de combinaciones de parámetros de entrenamiento es 250


In [36]:
#usamos la recomendación de probar con el 10%

cbst = Cubist()
cbst_random = RandomizedSearchCV(estimator = cbst,
                               param_distributions = random_grid,
                               n_iter = 5,
                               cv = 3,          # Validación cruzada 3-fold
                               verbose=2,
                               random_state=0,
                               n_jobs = -1      # Paralelizar en todos los cores disponibles
                               )
cbst_random.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


RandomizedSearchCV(cv=3, estimator=Cubist(), n_iter=5, n_jobs=-1,
                   param_distributions={'extrapolation': [0.01, 0.03, 0.05,
                                                          0.07, 0.09],
                                        'n_committees': [1, 2, 5, 10, 15],
                                        'n_rules': [200, 288, 377, 466, 555,
                                                    644, 733, 822, 911, 1000],
                                        'neighbors': [1]},
                   random_state=0, verbose=2)

In [37]:
cbst_random_best = cbst_random.best_estimator_

print('Los hiperparámetros del mejor modelo son:')
pprint(cbst_random.best_params_)
print()

print('Exactitud luego de búsqueda aleatoria en entrenamiento:', cbst_random_best.score(X_train, y_train))
print('Exactitud luego de búsqueda aleatoria en validación:', cbst_random_best.score(X_test, y_test))

Los hiperparámetros del mejor modelo son:
{'extrapolation': 0.03, 'n_committees': 15, 'n_rules': 377, 'neighbors': 1}

Exactitud luego de búsqueda aleatoria en entrenamiento: 0.999999999999999
Exactitud luego de búsqueda aleatoria en validación: 0.9999029658252098


**Uso de la librería OPTUNA para encontrar los mejores hiperparámetros**

Estimamos los mejores hiperparámetros usando la librería OPTUNA sugerida en el plantemiento del proyecto

In [42]:
import optuna
import optuna.visualization as vis

# Para trabajar con optuna, debemos definir la función objetivo que luego le diremos a optuna que debe minimizar (según nuestro indicador escogido RMSE)
def objective(trial):
  # Dejamos que optuna sugiera los hiperparámetros iniciales
  n_rules = trial.suggest_int("n_rules", 200, 1000)
  n_committees = trial.suggest_int("n_committees", 1, 15)
  neighbors = trial.suggest_int("neighbors", 1, 9)
  extrapolation = trial.suggest_float("extrapolation",0.01, 0.09)

  #Definimos el modelo Cubist a utilizar para la optimización
  cbst = Cubist(n_rules= n_rules, n_committees=n_committees, neighbors=neighbors, extrapolation=extrapolation)
  cbst.fit(X_train, y_train)

  # Make predictions and calculate RMSE
  y_pred = model.predict(X_test)
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))
  mae = mean_absolute_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)

  # Return MAE
  return mae

In [39]:
# Creamos el "objeto de estudio", como llama Optuna
study = optuna.create_study(direction="minimize")

# Ejecutamos el proceso en si de optimización, le pasamos la función objetivo definida en el paso previo
study.optimize(objective, n_trials=10, show_progress_bar=True)

[I 2024-06-17 04:56:35,265] A new study created in memory with name: no-name-84cff8a8-ad3b-482f-889c-fd6896fde123


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2024-06-17 04:57:25,765] Trial 0 finished with value: 0.06420718489852668 and parameters: {'n_rules': 212, 'n_committees': 15, 'neighbors': 3, 'extrapolation': 0.061987850705431245}. Best is trial 0 with value: 0.06420718489852668.
[I 2024-06-17 04:58:21,439] Trial 1 finished with value: 0.06420718489852668 and parameters: {'n_rules': 408, 'n_committees': 13, 'neighbors': 2, 'extrapolation': 0.026152889554376398}. Best is trial 0 with value: 0.06420718489852668.
[I 2024-06-17 04:59:22,117] Trial 2 finished with value: 0.06420718489852668 and parameters: {'n_rules': 504, 'n_committees': 14, 'neighbors': 4, 'extrapolation': 0.06774192295025387}. Best is trial 0 with value: 0.06420718489852668.
[I 2024-06-17 05:00:23,550] Trial 3 finished with value: 0.06420718489852668 and parameters: {'n_rules': 603, 'n_committees': 14, 'neighbors': 9, 'extrapolation': 0.025234673151967577}. Best is trial 0 with value: 0.06420718489852668.
[I 2024-06-17 05:00:42,748] Trial 4 finished with value: 0.06

In [40]:
# Imprimimos los resultados del estudio con optuna
print("Best trial:", study.best_trial)
print("Best hyperparameters:", study.best_params)

Best trial: FrozenTrial(number=0, state=TrialState.COMPLETE, values=[0.06420718489852668], datetime_start=datetime.datetime(2024, 6, 17, 4, 56, 35, 297681), datetime_complete=datetime.datetime(2024, 6, 17, 4, 57, 25, 764976), params={'n_rules': 212, 'n_committees': 15, 'neighbors': 3, 'extrapolation': 0.061987850705431245}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_rules': IntDistribution(high=1000, log=False, low=200, step=1), 'n_committees': IntDistribution(high=15, log=False, low=1, step=1), 'neighbors': IntDistribution(high=9, log=False, low=1, step=1), 'extrapolation': FloatDistribution(high=0.09, log=False, low=0.01, step=None)}, trial_id=0, value=None)
Best hyperparameters: {'n_rules': 212, 'n_committees': 15, 'neighbors': 3, 'extrapolation': 0.061987850705431245}


In [55]:
#Utilizamos algunos gráficos utiles de la librería optuna
# Plotear el historial de la optimización realizada
vis.plot_optimization_history(study)

# Plotear tipo 'slice'
#vis.plot_slice(study, params=["n_rules", "n_committees"])

# Ploteo tipo 'contorno'
#vis.plot_contour(study, params=["neighbors", "extrapolation"])

# Plotear parallel_coordinate
vis.plot_parallel_coordinate(study)

In [56]:
optuna.visualization.plot_optimization_history(study)